In [ ]:
import socket
import json
import requests

#Note to self, remove IP argument and check impact on class

#This iniatializes the attributes of the class
class AuctionHouse(object):	
	def __init__(self, Realmname, APICode, IP):
		self.Realmname = Realmname
		self.APICode = APICode
		self.IP = IP
	
	#Class function which obtains the users IP adress which is fed to the Blizzard endpoint as a required API argument
	def get_Host_name_IP(self):
		try:
			host_ip = requests.get('http://ip.42.pl/raw').text
			return host_ip
		except:
			print("Unable to get Hostname and IP")
	
	#The first step of the Blizzard API is to get the latest realm upload. This gets the latest upload URL and and returns it in this function 
	def Get_Data_URL(self):
		try:
			Realm = self.Realmname
			API = self.APICode
			baseurl =f'https://us.api.battle.net/wow/auction/data/{Realm}?locale=en_US&apikey={API}'
			headers = {'X-Originating-IP': str(AuctionHouse.get_Host_name_IP('self'))}
			r = requests.get(baseurl, headers=headers)
			JsonData = r.json()
			for files in JsonData['files']:
				return files['url']
		except:
			print(r)
	#This class function utilizes the URL obttained from Get_Data_URL. All of the selected realms AH data is loaded to a Json dictionary 
	#If there is an error the URL status is returned (i.e. 404 or whatever is applicable)
	def GetFullAH(self):
		try:
			url = str(self.Get_Data_URL())
			r = requests.get(url)
			JsonData = r.json()
			return JsonData
		except:
			print(r)

In [ ]:
class1 = AuctionHouse("moon guard", "ENTER_YOUR_API_HERE", AuctionHouse.get_Host_name_IP("self"))


#This loads the AuctionHouse data to a Json Dictionary and a dataframe.
#Pandas is not a requirement but is helpful for analysis.
JsonRawData =  class1.GetFullAH()
df = pd.DataFrame(JsonRawData['auctions'])
print(df.to_string())

In [ ]:
#Teste
import requests
import json

# Replace these variables with appropriate values
region = "us"
connected_realm_id = 1146
namespace = "dynamic-us"
locale = "en_US"
access_token = "YOUR_ACCESS_TOKEN"  # Replace with your actual access token

#reams_id list


The World of Warcraft API supports the following locales:

Locale	Value
English (United States)	en_US
Spanish (Mexico)	es_MX
Portuguese	pt_BR
German	de_DE
English (Great Britain)	en_GB
Spanish (Spain)	es_ES
French	fr_FR
Italian	it_IT
Russian	ru_RU
Korean	ko_KR
Chinese (Traditional)	zh_TW
Chinese (Simplified)	zh_CN



In [ ]:
# API endpoint
url = f"https://{region}.api.blizzard.com/data/wow/connected-realm/{connected_realm_id}/auctions"

# Headers and parameters for the request
headers = {
    "Authorization": f"Bearer {access_token}"
}
params = {
    "namespace": namespace,
    "locale": locale
}

def get_auction_data():
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

def main():
    auction_data = get_auction_data()
    
    if auction_data:
        # Print the auction data to a JSON file for better readability
        with open("auction_data.json", "w") as json_file:
            json.dump(auction_data, json_file, indent=4)
        
        # Calculate total number of auctions and total volume
        total_auctions = len(auction_data.get("auctions", []))
        total_volume = sum(auction["quantity"] for auction in auction_data.get("auctions", []))
        
        print(f"Total number of auctions: {total_auctions}")
        print(f"Total volume of negotiations: {total_volume}")

if __name__ == "__main__":
    main()
